In [49]:
import configparser
import csv
import random
config = configparser.ConfigParser()
a = config.read('config.ini')

In [51]:
def jitter(mean, std=0.2):
    sigma = mean*std
    k=random.gauss(mean, sigma)
    return k

def clockDeltaTime(t):
    exec(config.get("clock","clock_delay_function"))
    return locals()['delta']

# print(clockDeltaTime(12*60*60)) # inputting is seconds.

In [2]:
def networkMessageDelay(transPackageSize, transBandWidth, propagationDistance, propagationLoss,propagationSpeed=2 ):
    transTime =  (transPackageSize *8 * (1+ propagationLoss)) / (transBandWidth* 1.e6)
    propagationTime = 1000*propagationDistance / ( propagationSpeed * 1.e8)
    total=0.0
    subTotal=0.0
    subTotal= transTime + propagationTime
    total =subTotal+jitter(subTotal)
    return total  # return seconds

In [4]:
def serverOperatorTime(hardwareDelay=0.1,softwareDealy=0.25):
    h=0.0
    s=0.0
    h += jitter(hardwareDelay)
    s += jitter(softwareDealy)
    return h+s
 

def curTimestamp():
    from datetime import datetime, timezone
    #print(datetime.now().timestamp())
    return datetime.now().timestamp()

In [4]:
#c0->r1->r2->r3->c1

#package_size: 2000 Byte

#transBandWidth: 1Gb, 10GB, 10Gb, 11MB

#distance: 0.1km, 2km, 8km, 0.1km

#propagationSpeed: 2 * 1.e8

#Signal_loss: 0.01%, 0.01%, 0.15%, 0.5%


#Travel_Time_from_c0_to_c1:

def Message_Time_from_c0_to_c1():
    t1=networkMessageDelay(2000, 1024, 0.1, 0.01)
    # print("t1=%.12f"% t1, "seconds")
    
    t2=networkMessageDelay(2000, 10240, 2, 0.01)
    # print("t2=%.12f"% t2, "seconds")
    
    t3=networkMessageDelay(2000, 10240, 8, 0.15)
    # print("t3=%.12f"% t3, "seconds")
    
    t4=networkMessageDelay(2000, 11, 0.1, 0.5)
    # print("t4=%.12f"% t4, "seconds")
    
    totalTime= t1+t2+t3+t4
    return totalTime 

In [1]:
def purdueCampus():
    c1_send= curTimestamp()-jitter(100)
    c1_to_c0 =  Message_Time_from_c0_to_c1()
    c0_server =  curTimestamp()
    c0_to_c1 =  Message_Time_from_c0_to_c1()
    msgDelay =  c1_to_c0 +c0_to_c1
    #c1_recv=  c1_send + clockDeltaTime(msgDelay)
    halfMsgDelay = msgDelay/2
    print("\n\n'c1'=%.9f" % c1_send, "seconds")
    print(" c1 =%.9f" % (halfMsgDelay +c0_server), "seconds")
    print(" c0 =%.9f" % (c0_server + c0_to_c1), "seconds")